In [61]:
import numpy as np

# Sigmoid function

In [74]:
def sigmoid_forward(x):
    return 1 / (1 + np.exp(-x))

In [76]:
def sigmoid_backward(x):
    return sigmoid_forward(x) * (1 - sigmoid_forward(x))

# ReLU function

In [78]:
def relu_forward(x):
    return np.maximum(0, x)

In [84]:
def relu_backward(x):
    return 1 if x > 0 else 0

# Softmax function

In [82]:
def softmax_forward(vector):
    e = np.exp(vector)
    return e / np.sum(e)

In [80]:
def softmax_backward(vector, y):
    p = softmax_forward(vector)
    return p - y

# Implemented Neural network

In [59]:
class NeuralNetwork:
    def __init__(self):
        pass

    def sigmoid_forward(self, x):
        return 1 / (1 + np.exp(-x))

    def sigmoid_backward(self, x):
        return sigmoid_forward(x) * (1 - sigmoid_forward(x))

    def relu_forward(self, x):
        return np.maximum(0, x)

    def relu_backward(self, x):
        return 1 if x > 0 else 0

    def softmax_forward(self, vector):
        e = np.exp(vector)
        return e / np.sum(e)

    def softmax_backward(self, vector, y):
        p = softmax_forward(vector)
        return p - y